In [74]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [75]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [76]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:]
X_dev = X_dev / 255.

data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:]
X_train = X_train / 255.
_,m_train = X_train.shape

In [78]:
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [79]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2


def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [80]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [84]:
W1, B1, W2, B2 = gradient_descent(X_train, Y_train, 0.1, 500)

Iteration:  0
[2 0 2 ... 0 2 1] [2 0 3 ... 0 5 6]
0.12141463414634146
Iteration:  10
[7 0 2 ... 0 7 8] [2 0 3 ... 0 5 6]
0.21421951219512195
Iteration:  20
[7 0 2 ... 0 8 8] [2 0 3 ... 0 5 6]
0.2627317073170732
Iteration:  30
[7 0 2 ... 0 8 4] [2 0 3 ... 0 5 6]
0.3298780487804878
Iteration:  40
[7 0 2 ... 0 3 4] [2 0 3 ... 0 5 6]
0.40426829268292686
Iteration:  50
[3 0 2 ... 0 3 4] [2 0 3 ... 0 5 6]
0.4675365853658537
Iteration:  60
[3 0 8 ... 0 3 4] [2 0 3 ... 0 5 6]
0.5103658536585366
Iteration:  70
[3 0 8 ... 0 0 4] [2 0 3 ... 0 5 6]
0.5472439024390244
Iteration:  80
[3 0 8 ... 0 0 4] [2 0 3 ... 0 5 6]
0.5769756097560975
Iteration:  90
[3 0 8 ... 0 0 4] [2 0 3 ... 0 5 6]
0.6026585365853658
Iteration:  100
[3 0 8 ... 0 0 6] [2 0 3 ... 0 5 6]
0.6243414634146341
Iteration:  110
[3 0 3 ... 0 0 6] [2 0 3 ... 0 5 6]
0.6426585365853659
Iteration:  120
[3 0 3 ... 0 0 6] [2 0 3 ... 0 5 6]
0.6583658536585366
Iteration:  130
[3 0 3 ... 0 0 6] [2 0 3 ... 0 5 6]
0.6721219512195122
Iteration:  14